In [1]:
from newsplease import NewsPlease
import pandas as pd
import nltk
from tqdm import tnrange
import re

In [3]:
# make lookup number of days in month
lengths = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30 ,31]
lengths = [x + 1 for x in lengths]
def load_month(month):
    month_str = str(month).zfill(2)
    df_paths = ["2018{}{}.csv".format(month_str, str(x).zfill(2)) for x in range(1, lengths[month - 1])]
    df_list = [pd.read_csv("data/" + x) for x in df_paths]
    return df_list

In [9]:
relevant_words = ['land', 'forest', 'agriculture', 'farm', 'farmer', 'plantation', 'agrarian',
                  'smallholder', 'grazing', 'development', 'habitat', 'resource', 'cattle',
                  'livestock', 'kill', 'dead', 'airport', 'aluminum', 'mining', 'agro', 'dam',
                  'road', 'infrastructure', 'transmission', 'conservation', 'settlement', 'displace',
                  'exile', 'caste', 'conflict', 'relocation', 'village', 'encroach', 'fertilizer', 'mine',
                  'water', 'cow', 'cattle', 'appropriation', 'appropriate', 'protest', 'environmental', 'pollution',
                  'copper', 'iron', 'timber', 'acre', 'hectare', ]

def find_link_to_scrape(df):
    df['to_scrape'] = ''
    df['title'] = ''
    for i in range(len(df)):
        links = df['SOURCEURL'][i]
        l = re.findall(r'\w+(?:-\w+)+', links)
        if l:
            title = max(l, key = len)
            title = title.replace('-', ' ')
            df['title'][i] = title
            if any(word in title for word in relevant_words):
                df['to_scrape'][i] = str(links)
    return df

In [10]:
def combine_days(dfs):
    df_parsed = [find_link_to_scrape(dfs[x]) for x in tnrange(len(dfs))]
    df_month = pd.concat(df_parsed)
    df_subs = df_month[df_month['to_scrape'] != '']
    df_subs = df_subs.reset_index()
    return df_subs

In [11]:
for i in range(1, 11):
    month = load_month(i)
    df = combine_days(month)
    df.to_csv("interim/{}.csv".format(str(i).zfill(2)))

/Users/john.brandt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/john.brandt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Test scraping one month

In [31]:
import pickle
import os

month = '01'

def save_obj(obj, name):
    'Helper function using pickle to save and load objects'
    with open('../data/metadata/matching/' + name + '.pkl', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

df = pd.read_csv("../data/metadata/variables/{}.csv".format(month))
urls = df['to_scrape'].unique()

# create a dictionary mapping urls to the month's csv
mapping_dictionary = {}
for i, val in enumerate(urls):
    match = df.index[df['to_scrape'] == urls[i]].tolist()
    mapping_dictionary[i] = match
    
save_obj(mapping_dictionary, month)

/Users/john.brandt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (17,24,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
mapping_dictionary

{0: [0, 1, 2, 3, 4],
 1: [5, 6],
 2: [7, 11],
 3: [8],
 4: [9, 10, 12, 13],
 5: [14, 15],
 6: [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 7: [30, 31],
 8: [32, 33],
 9: [34],
 10: [35, 38, 39, 40],
 11: [36, 41, 42, 43, 44, 45],
 12: [37],
 13: [46],
 14: [47, 48],
 15: [49, 51],
 16: [50, 52, 53, 54],
 17: [55, 56, 57, 58, 59, 60, 61],
 18: [62, 63, 64],
 19: [65, 66, 70, 71],
 20: [67],
 21: [68, 69],
 22: [72, 73],
 23: [74, 75],
 24: [76, 78],
 25: [77],
 26: [79, 80],
 27: [81],
 28: [82, 83, 91, 92, 95, 96, 97, 98, 99, 100, 101, 102],
 29: [84, 85, 86, 87, 88, 89, 90],
 30: [93, 94, 105, 106],
 31: [103, 104, 107, 108],
 32: [109],
 33: [110, 112, 114, 115, 116, 117, 118, 123, 124, 125, 126],
 34: [111, 113, 119, 120, 121, 122],
 35: [127, 128],
 36: [129],
 37: [130, 131],
 38: [132, 133],
 39: [134, 136],
 40: [135, 137, 138],
 41: [139],
 42: [140],
 43: [141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  160,
  161,
  162,
  163,
  164,
  165

In [ ]:
def save_obj(obj, name):
    'Helper function using pickle to save and load objects'
    with open('text/' + str(month) + '/' + name + '.pkl', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import multiprocessing

month = 11

if not os.path.exists("text/{}".format(str(month))):
    os.makedirs("text/{}".format(str(month)))

    
def download_url(i):
    try:
        article = NewsPlease.from_url(urls[i])
        save_obj(article, str(i).zfill(5))
        return 1
    except Exception as ex:
        print(i, ex)
        return 0

potential = range(0, len(urls))
existing = os.listdir("text/{}/".format(str(month)))
existing = [int(x[:5]) for x in existing if ".DS" not in x]
potential = [x for x in potential if x not in existing]
print(len(potential))
pool = multiprocessing.Pool(16)
zip(*pool.map(download_url, potential))


In [ ]:
pool.close()
pool.join()